In [111]:
from transformers import pipeline
import pandas as pd
#import defaultdict module  
from collections import defaultdict
import string

In [94]:
def preprocess(sentence):
    return sentence.strip().lower()

In [112]:
def is_word(word):
    return not all(c.isdigit() or c in string.punctuation for c in word)

In [95]:
relations = pd.read_csv("../data/raw/_SELECT_CUI1_AUI1_c1_STR_REL_RELA_CUI2_AUI2_c2_STR_FROM_umls_MRR_202307141231.csv").applymap(preprocess)

In [96]:
analogies = defaultdict(list)
# iterate for every group in a groupby object
for name, group in relations.groupby("RELA"):
    #shuffle group dataframe
    group = group.sample(frac=1)
    #iterate every two rows over group dataframe   
    for i in range(0, len(group), 2):
        #get the first row
        row1 = group.iloc[i]
        row2 = group[group["STR.1"] != row1["STR.1"]].sample(1).iloc[0]
        question = "{} es a {} como {} es a".format(row1["STR"], row1["STR.1"], row2["STR"])
        answer = row2["STR.1"]
        analogies[name].append({"question": question, "answer": answer})

In [64]:
pipe = pipeline("fill-mask", model="PlanTL-GOB-ES/roberta-base-biomedical-clinical-es", device=0)

In [122]:
K = 20
results = defaultdict(list)
for rela, current_analogies in analogies.items():
    print(rela)
    for analogy in current_analogies:
        i = 2
        predicted_words = []
        while len(predicted_words) < K:
            predictions = pipe(f"{analogy['question']} <mask>.", top_k=i*K)
            predicted_words = [preprocess(prediction["token_str"]) for prediction in predictions if is_word(prediction["token_str"])][:K]
            i += 1
        results[rela].append(predicted_words)

associated_finding_of
causative_agent_of
cause_of
direct_device_of
finding_method_of
focus_of
has_modification
has_occurrence


TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [147]:
accuracies = defaultdict(list)

In [148]:
def accuracy_at_k(true, predicted, k):
    result = []
    for i in range(len(predicted)):
        if true[i] in predicted[i][:k]:
            result.append(1)
        else:
            result.append(0)
    return sum(result)/len(result)

In [149]:
for analogy, result in zip(analogies.items(), results.items()):
    true = [x["answer"] for x in analogy[1]]
    predicted = result[1]
    for k in range(1, K+1):
        accuracies[analogy[0]].append(accuracy_at_k(true, predicted, k))

In [150]:
accuracies

defaultdict(list,
            {'associated_finding_of': [0.06341463414634146,
              0.16097560975609757,
              0.1902439024390244,
              0.24878048780487805,
              0.28292682926829266,
              0.2926829268292683,
              0.2975609756097561,
              0.3121951219512195,
              0.32195121951219513,
              0.33170731707317075,
              0.34146341463414637,
              0.3463414634146341,
              0.3463414634146341,
              0.3463414634146341,
              0.35121951219512193,
              0.37073170731707317,
              0.37073170731707317,
              0.37073170731707317,
              0.375609756097561,
              0.375609756097561],
             'causative_agent_of': [0.043054611375481534,
              0.067754362111942,
              0.08135055517788353,
              0.08656242918649446,
              0.09313392250169952,
              0.09879900294584183,
              0.10197144799456152,
 

In [123]:
for group, accuracy in accuracies.items():
    print(group, accuracy)

associated_finding_of 0.28292682926829266
causative_agent_of 0.09313392250169952
cause_of 0.00881057268722467
direct_device_of 0.020242914979757085
finding_method_of 0.6461538461538462
focus_of 0.04152249134948097
has_modification 0.07011494252873564
has_occurrence 0.01762307020099039
